In [2]:
import pickle
import numpy
import math
import matplotlib
from matplotlib.colors import LogNorm
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter
import numpy as np
from collections import OrderedDict
#import plotly.plotly as py
#import plotly.tools as tls

import matplotlib.pyplot as plt

def plot_app_box(app_dict, y_lst):
    plt.gcf().clear()
    plt.ylim([min(y_lst), max(y_lst)*10])
    
    x_lst = app_dict.keys()
    bw_lst = app_dict.values()
    plt.xlabel('App ID')
    plt.ylabel('Bandwidth (GB/s)')
    plt.yscale('log')
    plt.boxplot(bw_lst, '', labels = x_lst)
    plt.xticks(rotation=90)
    title = "app_box"
    plt.savefig(title+'.jpg', format = 'jpg', bbox_inches='tight')
#    plt.xscale('log')
#    plt.title('TestBW')
    plt.legend()

def get_name(name):
    if name == "cesm.exe":
        return "Climate1"
    if "Nyx" in name:
        return "Cosmology2"
    if name == "hacc_tpm":
        return "Cosmology1"
    if name == "pw.x":
        return "Quantum1"
    if name == "s3d.x":
        return "Combustion1"
    if name == "vasp_std":
        return "Quantum2"
    if name == "elm_6f":
        return "Physics1"
    if name == "wrf.exe":
        return "Climate2"
    if name == "phoenix":
        return "Astro1"
    if name == "cgyro":
        return "Fusion1"
    if name == "gene_cori":
        return "Biology1"
    if "sgw.x" in name:
        return "Material2"
    if "pot" in name:
        return "Material4"
    if "delay" in name:
        return "Material1"
    if "tc.x" in name:
        return "Material3"
    return ""
    
tmp_str = "/global/cscratch1/sd/tengwang/miner1217/app_bw_ext.log"
save_fd = open(tmp_str, 'rb')
feature_vec = pickle.load(save_fd)
save_fd.close()

tmp_str = "/global/cscratch1/sd/tengwang/miner1217/app_cnt.log"
save_fd = open(tmp_str, 'rb')
cycle_dict = pickle.load(save_fd)
save_fd.close()

sorted_cycle_dict = sorted(cycle_dict.items(), key = lambda x:-x[1][0])


app_dict = OrderedDict()
app_cycle_dict = {}
for record in feature_vec:
    int_val = record[3]   
    if app_cycle_dict.get(int_val, -1) == -1:
        app_cycle_dict[int_val] = ([], cycle_dict[int_val][0])
    app_cycle_dict[int_val][0].append(float(record[2])/1024) 
#    print ("app:%s, cycle:%lf\n"%(int_val, cycle_dict[int_val][0]))

sorted_app_cycle_dict = sorted(app_cycle_dict.items(), key = lambda x:-x[1][1])
for record in sorted_app_cycle_dict:
    int_val = get_name(record[0])
    if int_val == "":
        int_val = record[0]
    if "Nyx3d" in int_val :
        int_val = "Nyx3d"
    if app_dict.get(int_val, -1) == -1:
        app_dict[int_val] = record[1][0]
    #app_dict[int_val].append(float(record[2])/1024)
    # print "appid:%d, name:%s, bw:%lf\n"%(int(record[0]), record[1], float(record[2]))
    
y_lst = [float(x[2])/1024 for x in feature_vec]
plot_app_box(app_dict, y_lst)
